# 更新对象
Weaviate 允许部分或全部对象更新。

In [1]:
import json
import weaviate
from weaviate.auth import AuthApiKey

# 连接到本地部署的 Weaviate
client = weaviate.Client(
    url="http://127.0.0.1:8080",
    auth_client_secret=AuthApiKey("WVF5YThaHlkYwhGUSmCRgsX3tD5ngdN8pkih")
)

## 更新对象属性
此操作仅替换指定属性的整个值，保留未指定的属性。提供集合名称、对象 ID 和要更新的属性。

如果您更新先前矢量化的属性的值，Weaviate 会自动重新矢量化该对象。这还会重新索引更新后的对象。

但是，如果您在集合定义中添加新属性，Weaviate 只会对新对象进行矢量化。定义新属性时，Weaviate 不会重新矢量化和重新索引现有对象，只有在更新现有属性时才会这样做。

In [2]:
client.data_object.get_by_id(
    uuid="3e8346aa-0c95-433b-8fb3-32aac342f8c0"
)

/opt/anaconda3/envs/llm/lib/python3.11/site-packages/weaviate/data/crud_data.py:533: DeprecationWarning: Weaviate Server version >= 1.14.x STRONGLY recommends using class namespaced APIs, please specify the `class_name` argument for this. The non-class namespaced APIs (None value for `class_name`) are going to be removed in the future versions of the Weaviate Server and Weaviate Python Client.
  warnings.warn(


{'class': 'JeopardyQuestion',
 'creationTimeUnix': 1744127483985,
 'id': '3e8346aa-0c95-433b-8fb3-32aac342f8c0',
 'lastUpdateTimeUnix': 1744127483985,
 'properties': {'answer': 'Weaviate',
  'question': 'This vector DB is OSS and supports automatic property type inference on import'},
 'vectorWeights': None}

In [4]:
### V3
uuid = "3e8346aa-0c95-433b-8fb3-32aac342f8c0"  # replace with the id of the object you want to update
client.data_object.update(
    uuid=uuid,
    class_name="JeopardyQuestion",
    data_object={
        "answer": "Weaviate Weaviate" ,
    },
)

In [5]:
client.data_object.get_by_id(
    uuid="3e8346aa-0c95-433b-8fb3-32aac342f8c0"
)

{'class': 'JeopardyQuestion',
 'creationTimeUnix': 1744127483985,
 'id': '3e8346aa-0c95-433b-8fb3-32aac342f8c0',
 'lastUpdateTimeUnix': 1744164920139,
 'properties': {'answer': 'Weaviate Weaviate',
  'question': 'This vector DB is OSS and supports automatic property type inference on import'},
 'vectorWeights': None}

In [ ]:
### V4
uuid = "..."  # replace with the id of the object you want to update

jeopardy = client.collections.get("JeopardyQuestion")
jeopardy.data.update(
    uuid=uuid,
    properties={
        "points": 100,
    }
)

## 更新对象向量
对象向量也可以像属性一样进行更新。对于命名向量，以字典/映射的形式提供数据，类似于对象创建。

In [ ]:
## V3
client.data_object.update(
    uuid=uuid,
    class_name="JeopardyQuestion",
    data_object={
        "points": 100,
    },
    vector=[0.12345] * 1536
)

In [ ]:
### V4
jeopardy = client.collections.get("JeopardyQuestion")
jeopardy.data.update(
    uuid=uuid,
    properties={
        "points": 100,
    },
    vector=[0.12345] * 1536
)

## 替换整个对象
通过提供集合名称、id 和新对象可以替换整个对象。

In [6]:
client.data_object.replace(
    uuid=uuid,
    class_name="JeopardyQuestion",
    data_object={
        "answer": "Replaced",
        # The other properties will be deleted
    },
)

In [7]:
client.data_object.get_by_id(
    uuid="3e8346aa-0c95-433b-8fb3-32aac342f8c0"
)

{'class': 'JeopardyQuestion',
 'creationTimeUnix': 1744127483985,
 'id': '3e8346aa-0c95-433b-8fb3-32aac342f8c0',
 'lastUpdateTimeUnix': 1744165009387,
 'properties': {'answer': 'Replaced'},
 'vectorWeights': None}

In [ ]:
### V4
jeopardy = client.collections.get("JeopardyQuestion")
jeopardy.data.replace(
    uuid=uuid,
    properties={
        "answer": "Replaced",
        # The other properties will be deleted
    },
)

## 删除属性
尚不支持删除或更新集合定义中的属性。

""在对象级别，您可以用已删除这些属性或设置为文本属性的副本替换该对象。

In [ ]:
### V3
from typing import List
from weaviate import Client

def del_props(client: Client, uuid: str, class_name: str, prop_names: List[str]) -> None:
    object_data = client.data_object.get(uuid, class_name=class_name)
    for prop_name in prop_names:
        if prop_name in object_data["properties"]:
            del object_data["properties"][prop_name]
    client.data_object.replace(object_data["properties"], class_name, uuid)


uuid = "..."  # replace with the id of the object you want to delete properties from
del_props(client, uuid, "JeopardyQuestion", ["answer"])

In [ ]:
## V4

from typing import List
from weaviate import WeaviateClient

def del_props(client: WeaviateClient, uuid_to_update: str, collection_name: str, prop_names: List[str]) -> None:
    collection = client.collections.get(collection_name)

    # fetch the object to update
    object_data = collection.query.fetch_object_by_id(uuid_to_update)
    properties_to_update = object_data.properties

    # remove unwanted properties
    for prop_name in prop_names:
        if prop_name in properties_to_update:
            del properties_to_update[prop_name]

    # replace the properties
    collection.data.replace(
        uuid=uuid_to_update,
        properties=properties_to_update
    )


uuid = "..."  # replace with the id of the object you want to delete properties from
del_props(client, uuid, "JeopardyQuestion", ["answer"])